#### Basic Workflow of retrivel part of `RAG`
`ingest documents -> split into chunks -> embed chunks -> make a query -> embed the query -> compare query embedding to chunk embeddings`

In [2]:
import random
import numpy as np
import pandas as pd
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [36]:
# import text_chunks_and_embeddings file
text_chunks_and_embedding_df=pd.read_csv(r".\data\text_chunks_and_embeddings.csv")

# converting embedding column back to np.array([])
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
# converting to dictionaries
pages_and_chunks=text_chunks_and_embedding_df.to_dict(orient="records")
# converting embeedint to torch tensor with float32 datatype
embeddings=torch.tensor(text_chunks_and_embedding_df["embedding"].tolist(),dtype=torch.float32)
embeddings.shape

torch.Size([1670, 768])

Now that we have embeddings for our data, now is time to instantiate a model for the "query" embeddings which would we use to find the documents/sentences in our data/corpus similar to query

In [39]:
from sentence_transformers import util, SentenceTransformer
embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                    device=device)

In [1]:
# Cosine Similarity

query = "What is the importance of a healthy diet"
query_embedding=embedding_model.encode(query,convert_to_tensor=True )
#similarity search based on cosine similarity
cos_sim_scores=util.cos_sim(a=query_embedding,b=embeddings)
# score_indices=torch.argsort(cos_sim_scores,keepdim=True)

# top k scores
topK=torch.topk(cos_sim_scores,k=4)



NameError: name 'embedding_model' is not defined

In [83]:
# Define helper function to print wrapped text 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

# Top k retrievals
s=0
for idx in topK.indices[-1]:
    print("Score: ", topK.values[0][s].item())
    s+=1
    print_wrapped(text_chunks_and_embedding_df["sentence_chunk"][idx.item()])
    print()


Score:  0.6641526222229004
Achieving a Healthy Diet UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN
NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM Achieving a healthy diet is a
matter of balancing the quality and quantity of food that is eaten. There are
five key factors that make up a healthful diet: 1. A diet must be adequate, by
providing sufficient amounts of each essential nutrient, as well as fiber and
adequate calories.2. A balanced diet results when you do not consume one
nutrient at the expense of another, but rather get appropriate amounts of all
nutrients.3. Calorie control is necessary so that the amount of energy you get
from the nutrients you consume equals the amount of energy you expend during
your day’s activities.4. Moderation means not eating to the extremes, neither
too much nor too little.5.

Score:  0.6617240905761719
Variety refers to consuming different foods from within each of the food groups
on a regular basis. A healthy diet is one that favors whole foo

In [2]:
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

AssertionError: Torch not compiled with CUDA enabled